In [16]:
# Install Miniconda
# https://docs.conda.io/en/latest/miniconda.html

In [17]:
# Add conda channels
! conda config --add channels defaults
! conda config --add channels bioconda
! conda config --add channels conda-forge

In [18]:
# Install Gromacs
! printf "y" | conda install -c bioconda gromacs

Solving environment: done

# All requested packages already installed.



In [24]:
# Install additional libraries
! printf "y" | conda install numpy pandas matplotlib
! printf "y" | conda install nglview mdanalysis
! printf "y" | pip install GromacsWrapper pytraj

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [20]:
# Initialize and enable NGLview
! jupyter-nbextension enable --py --sys-prefix widgetsnbextension
! jupyter-nbextension enable nglview --py --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Enabling notebook extension nglview-js-widgets/extension...
      - Validating: OK


In [ ]:
# numpy and matplotlib are required for result analysis and plotting

In [21]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# pytraj and nglview are required for trajectory analysis and visualization

In [25]:
import pytraj as pt
import nglview as nv